In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

In [ ]:
df=pd.read_excel('Data_Train.xlsx')

In [ ]:
df.dropna(inplace=True)
df['Duration']=df['Duration'].str.replace('h','*60').str.replace(' ','+').str.replace('m','*1').apply(eval)
df['Duration']=pd.to_numeric(df['Duration'])
df['Date_of_Journey']=pd.to_datetime(df['Date_of_Journey'])
df['Dep_Time']=pd.to_datetime(df['Dep_Time'],format='%H:%M').dt.time
df['weekday']=df[['Date_of_Journey']].apply(lambda x:x.dt.day_name())
df["month"] = df['Date_of_Journey'].map(lambda x: x.month_name())
df['Dep_Time']=df['Dep_Time'].apply(lambda x:x.hour)
df.drop(['Route','Arrival_Time','Date_of_Journey'],axis=1,inplace=True)
df['Dep_Time']=pd.to_numeric(df['Dep_Time'])
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
var_mod = ['Airline','Source','Destination','Additional_Info','Total_Stops','weekday','month']
le = LabelEncoder()
for i in var_mod:
    df[i] = le.fit_transform(df[i])
x=df.drop('Price',axis=1)
y=df['Price']

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor

In [ ]:
regg=[LinearRegression(),RandomForestRegressor(),GradientBoostingRegressor(),ExtraTreesRegressor(),SVR(),DecisionTreeRegressor()]

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
from sklearn.metrics import make_scorer

In [ ]:
mean=[]
std=[]
for i in regg:
    cvs=cross_val_score(i,x,y,cv=5,scoring=make_scorer(r2_score),verbose=1,n_jobs=-1)
    mean.append(np.mean(cvs))
    std.append(np.std(cvs))

In [ ]:
mean


In [ ]:
std

In [ ]:
for i in range(6):
    print(regg[i].__class__.__name__,':',mean[i])

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
p_lr={}
p_rfr={"n_estimators" : [10,20,30],"max_features" : ["auto", "sqrt", "log2"],
            "min_samples_split" : [2,4,8],
            "bootstrap": [True, False]}
p_svr={'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001]}
p_gbr={'n_estimators': [80,150],
        'max_features': [20,15],
	    'max_depth': [8,10],
        'learning_rate': [0.1,0.05,0.01]}
p_etr={'n_estimators': [80,150],
        'max_features': [20,15],
      "min_samples_split" : [2,4,8]}

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=101)

In [ ]:
g_lr=GridSearchCV(LinearRegression(),param_grid=p_lr,scoring=make_scorer(r2_score),cv=5,refit=True)

In [ ]:
g_lr.fit(x_train,y_train)

In [ ]:
g_rfr=GridSearchCV(RandomForestRegressor(),param_grid=p_rfr,scoring=make_scorer(r2_score),cv=5,refit=True)

In [ ]:
g_rfr.fit(x_train,y_train)

In [ ]:
g_rfr.best_estimator_

In [ ]:
g_rfr.best_params_

In [ ]:
g_svr=GridSearchCV(SVR(),param_grid=p_svr,scoring=make_scorer(r2_score),cv=5,refit=True)
g_svr.fit(x_train,y_train)

In [ ]:
g_etr=GridSearchCV(ExtraTreesRegressor(),param_grid=p_etr,scoring=make_scorer(r2_score),cv=5,refit=True)
g_etr.fit(x_train,y_train)

In [ ]:
g_gbr=GridSearchCV(GradientBoostingRegressor(),param_grid=p_gbr,scoring=make_scorer(r2_score),cv=5,refit=True)
g_gbr.fit(x_train,y_train)